#### Connect to mysql server

In [0]:
driver = "com.mysql.cj.jdbc.Driver"

database_host = "movie-server.mysql.database.azure.com"
database_port = "3306" # update if you use a non-default port
database_name = "movieservice"
table = "opening_weekend_gross"
user = "nguyentot"
password = "Tot2002@"

url = f"jdbc:mysql://{database_host}:{database_port}/{database_name}"

opening_gross_DF = (spark.read
  .format("jdbc")
  .option("driver", driver)
  .option("url", url)
  .option("dbtable", table)
  .option("user", user)
  .option("password", password)
  .load()
)

opening_gross_DF.show(5)

+---+-------------------+--------+
| id|     weekendEndDate|budgetId|
+---+-------------------+--------+
|  1|2001-12-30 00:00:00|    2202|
|  2|2003-11-02 00:00:00|    2203|
|  3|2010-05-30 00:00:00|    2204|
|  4|2016-09-18 00:00:00|    2207|
|  5|1991-04-28 00:00:00|    2209|
+---+-------------------+--------+
only showing top 5 rows



#### create mount to bronze


In [0]:
%fs ls /mnt/datalake/bronze/mysql

path,name,size,modificationTime
dbfs:/mnt/datalake/bronze/mysql/actor/,actor/,0,0
dbfs:/mnt/datalake/bronze/mysql/budget/,budget/,0,0
dbfs:/mnt/datalake/bronze/mysql/cast_to_movie/,cast_to_movie/,0,0
dbfs:/mnt/datalake/bronze/mysql/currency/,currency/,0,0
dbfs:/mnt/datalake/bronze/mysql/director_to_movie/,director_to_movie/,0,0
dbfs:/mnt/datalake/bronze/mysql/genre/,genre/,0,0
dbfs:/mnt/datalake/bronze/mysql/keyword/,keyword/,0,0
dbfs:/mnt/datalake/bronze/mysql/movie/,movie/,0,0
dbfs:/mnt/datalake/bronze/mysql/movie_to_genre/,movie_to_genre/,0,0
dbfs:/mnt/datalake/bronze/mysql/movie_to_keyword/,movie_to_keyword/,0,0


In [0]:
# Table name
opening_gross_table = "bronze.opening_gross"

# bronze location
opening_gross_bronzeLocation = "/mnt/datalake/bronze/mysql/opening_gross"

In [0]:
 
from delta.tables import *

#check if the bronze location contain the delta table
if (DeltaTable.isDeltaTable(spark, opening_gross_bronzeLocation)):
    # If Delta table exists, append the data

    # In the bronze zone, we do not do any filtering or remove duplicates
    opening_gross_DF.write.mode("append").format("delta").save(opening_gross_bronzeLocation)
else:
    # If no, save the file to bronzeLocation
    opening_gross_DF.write.mode("overwrite").format("delta").save(opening_gross_bronzeLocation)
    

In [0]:
# create the schema and table, if required
spark.sql("CREATE SCHEMA IF NOT EXISTS bronze")
spark.sql(f"CREATE EXTERNAL TABLE IF NOT EXISTS {opening_gross_table} USING delta LOCATION '{opening_gross_bronzeLocation}'")

# Note: Using spark.sql because we use f-string to retrieve the processedLocation

DataFrame[]

In [0]:
%sql
DESCRIBE HISTORY bronze.opening_gross;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,2023-11-21T15:51:35Z,1311347395691033,totnvt02@gmail.com,WRITE,"Map(mode -> Overwrite, statsOnLoad -> false, partitionBy -> [])",null,List(2204498048936660),1121-120918-uq53wl18,null,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 13803, numOutputBytes -> 141901)",null,Databricks-Runtime/13.3.x-scala2.12


In [0]:
# Maintenance for Delta table

# To optimized the performance of the Delta table, we need to excute 2 commands:
# 1. optimize(): Optimize the number of files used to store the data
# 2. vacuum(): Remove the old version of the data. This reduces the overhead but it limites the version we can go back to.
opening_gross_DataDelta = DeltaTable.forName(spark, opening_gross_table)

if opening_gross_DataDelta.history(30).filter("operation = 'VACUUM START'").count() == 0:
	opening_gross_DataDelta.optimize()
	opening_gross_DataDelta.vacuum()

In [0]:
%sql
DESCRIBE HISTORY bronze.opening_gross;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
2,2023-11-21T15:51:49Z,1311347395691033,totnvt02@gmail.com,VACUUM END,Map(status -> COMPLETED),null,List(2204498048936660),1121-120918-uq53wl18,1,SnapshotIsolation,true,"Map(numDeletedFiles -> 0, numVacuumedDirectories -> 1)",null,Databricks-Runtime/13.3.x-scala2.12
1,2023-11-21T15:51:46Z,1311347395691033,totnvt02@gmail.com,VACUUM START,"Map(retentionCheckEnabled -> true, defaultRetentionMillis -> 604800000)",null,List(2204498048936660),1121-120918-uq53wl18,0,SnapshotIsolation,true,"Map(numFilesToDelete -> 0, sizeOfDataToDelete -> 0)",null,Databricks-Runtime/13.3.x-scala2.12
0,2023-11-21T15:51:35Z,1311347395691033,totnvt02@gmail.com,WRITE,"Map(mode -> Overwrite, statsOnLoad -> false, partitionBy -> [])",null,List(2204498048936660),1121-120918-uq53wl18,null,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 13803, numOutputBytes -> 141901)",null,Databricks-Runtime/13.3.x-scala2.12
